In [1]:
from transformers import pipeline
import pandas as pd

/home/francisco/Documents/Francisco/Faculdade/7_Semestre/processamento_lingua_natural/tpFinal/tp_final_nlp/env_py/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-27 22:11:14.903217: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-27 22:11:14.958189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738026674.977279   21211 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one h

In [2]:
# load the data
data = pd.read_csv('./data/anime-dataset-2023.csv')


In [ ]:
# see column names
print(data.columns)

In [ ]:
data.head(2)

In [3]:
# get only the useful columns -> 'anime_id', 'Name', 'Genres', 'Synopsis'
data = data[['anime_id', 'Name', 'Genres', 'Synopsis']]

data.head()

,anime_id,Name,Genres,Synopsis
0,1,Cowboy Bebop,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi","Another day, another bounty—such is the life o..."
2,6,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...


In [4]:
# iterate over genres and preprocess them -> remove spaces, lowercase

data['Genres'] = data['Genres'].apply(lambda x: x.replace(' ', '').lower())

In [13]:
data.head(10)

,anime_id,Name,Genres,Synopsis
0,1,Cowboy Bebop,"action,awardwinning,sci-fi","Crime is timeless. By the year 2071, humanity ..."
1,5,Cowboy Bebop: Tengoku no Tobira,"action,sci-fi","Another day, another bounty—such is the life o..."
2,6,Trigun,"action,adventure,sci-fi","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,"action,drama,mystery,supernatural",Robin Sena is a powerful craft user drafted in...
4,8,Bouken Ou Beet,"adventure,fantasy,supernatural",It is the dark century and the people are suff...
5,15,Eyeshield 21,sports,"Shy, reserved, and small-statured, Deimon High..."
6,16,Hachimitsu to Clover,"comedy,drama,romance","Yuuta Takemoto, a sophomore at an arts college..."
7,17,Hungry Heart: Wild Striker,"comedy,sliceoflife,sports",As the younger brother of Japanese soccer star...
8,18,Initial D Fourth Stage,"action,drama",Takumi Fujiwara finally joins Ryousuke and Kei...
9,19,Monster,"drama,mystery,suspense","Dr. Kenzou Tenma, an elite neurosurgeon recent..."


In [5]:
# split data into 2 parts -> train and test
# train -> 80% of the data
# test -> 20% of the data
train = data.sample(frac=0.8, random_state=0)
test = data.drop(train.index)

In [6]:
data.shape, train.shape, test.shape

((24905, 4), (19924, 4), (4981, 4))

In [7]:
# predict the genre of the anime

# get the first anime synopsis
anime_synopsis = test['Synopsis'].iloc[0]
anime_name = test['Name'].iloc[0]
anime_genre = test['Genres'].iloc[0]

# all possible genres
all_genres = list(set(data['Genres'].str.cat(sep='|').split('|')))

print(all_genres)

genres = set()
for i in all_genres:
    genres_separated = i.replace(' ', '').split(',')
    for j in genres_separated:
        genres.add(j)

print(genres)

genres = list(genres)

['drama,fantasy,girlslove,romance', 'avantgarde,sports', 'comedy,sliceoflife', 'action,adventure,drama,sci-fi,supernatural', 'adventure,fantasy,romance,supernatural', 'avantgarde,drama,supernatural', 'boyslove,hentai', 'adventure,boyslove,comedy,fantasy', 'boyslove,erotica', 'action,adventure,sci-fi,supernatural', 'drama,sci-fi,suspense', 'drama,hentai', 'adventure,comedy,horror,romance,supernatural', 'adventure,comedy,gourmet', 'action,avantgarde,comedy,supernatural,ecchi', 'horror,supernatural,suspense', 'action,drama,mystery', 'adventure,drama,sliceoflife', 'action,comedy,fantasy,horror', 'horror,mystery,romance,supernatural', 'action,adventure,horror,mystery,sci-fi,suspense', 'fantasy,horror,supernatural,hentai', 'action,adventure,comedy,romance', 'action,adventure,suspense', 'boyslove,comedy,romance', 'fantasy,sci-fi,sliceoflife', 'action,comedy,romance,supernatural,hentai', 'adventure,drama,fantasy,romance,supernatural', 'action,comedy,fantasy,supernatural', 'action,comedy,girlsl

In [8]:
print(len(genres))

22


In [ ]:
print(anime_synopsis)

In [ ]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")  # change the model identifier here


In [ ]:
from tqdm import tqdm
import json

# Initialize results dictionary
results = {}

# Iterate over the test data with progress bar
for index, row in tqdm(test.iterrows(), total=test.shape[0], desc="Processing anime"):
    anime_id = row['anime_id']
    anime_synopsis = row['Synopsis']
    anime_name = row['Name']
    anime_genre = row['Genres']

    text = 'The anime is called ' + anime_name + '. The synopsis of the anime is: ' + anime_synopsis
    output = zeroshot_classifier(text, list(genres), multi_label=True)

    anime_genre_len = len(anime_genre.split(','))

    top_k_labels_predicted = output['labels'][0:anime_genre_len]

    results[anime_id] = {}

    for i in range(anime_genre_len):
        genre = anime_genre.split(',')[i].strip()
        results[anime_id][genre] = "error"
        if genre in top_k_labels_predicted:
            results[anime_id][genre] = "correct"

# Save the results to a JSON file
with open('results_based_model.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd

# Split into train and test sets
train = data.sample(frac=0.8, random_state=0)
test = data.drop(train.index)

# Extract all unique genres from the ENTIRE dataset
all_genres = set()
for genres in data['Genres'].dropna():
    all_genres.update(g.strip() for g in genres.split(','))  # Preserve spaces within genre names
unique_genres = sorted(all_genres)


# Verify the number of unique genres
print("Number of unique genres:", len(unique_genres)) 
print("Unique genres:", unique_genres)

# Preprocessing function
def preprocess(df):
    df['text'] = 'The synopsis of the anime is: ' + df['Synopsis']
    
    # Encode labels as binary vectors
    genre_to_id = {genre: idx for idx, genre in enumerate(unique_genres)}
    
    def encode_labels(genres):
        labels = [0] * len(unique_genres)
        if isinstance(genres, str):  # Check if genres is a string
            for genre in genres.split(','):
                labels[genre_to_id[genre.strip()]] = 1
        return labels
    
    df['labels'] = df['Genres'].apply(encode_labels)
    return df

# Preprocess train and test datasets
train = preprocess(train)
test = preprocess(test)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train[['text', 'labels']])
test_dataset = Dataset.from_pandas(test[['text', 'labels']])

# Tokenizer
model_name = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized = tokenizer(batch['text'], padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = batch["labels"]
    return tokenized

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Load Pre-trained Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_genres),  # Set to the number of unique genres (42)
    ignore_mismatched_sizes=True    # Ignore size mismatches
)

# Freeze pre-trained layers
for param in model.base_model.parameters():
    param.requires_grad = False

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    disable_tqdm=False, 
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train
trainer.train()

Number of unique genres: 22
Unique genres: ['action', 'adventure', 'avantgarde', 'awardwinning', 'boyslove', 'comedy', 'drama', 'ecchi', 'erotica', 'fantasy', 'girlslove', 'gourmet', 'hentai', 'horror', 'mystery', 'romance', 'sci-fi', 'sliceoflife', 'sports', 'supernatural', 'suspense', 'unknown']


Map: 100%|██████████| 4981/4981 [00:00<00:00, 6418.06 examples/s]
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/deberta-v3-large-zeroshot-v2.0 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([22]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([22, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/francisco/Documents/Francisco/Faculdade/7_Semestre/processamento_lingua_natural/tpFinal/tp_final_nlp/env_py/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save_pretrained('./model/fine-tuned-anime-genre-model')
tokenizer.save_pretrained('./model/fine-tuned-anime-genre-model')

In [ ]:
import json
from tqdm import tqdm

model_path = './model/fine-tuned-anime-genre-model'
zeroshot_classifier = pipeline("zero-shot-classification", model=model_path)

# Define genres
#genres = sorted(set(g.strip() for genre_list in data['Genres'].dropna() for g in genre_list.split(',')))

# Initialize results dictionary
results = {}

# Iterate over the test data with progress bar
for index, row in tqdm(test.iterrows(), total=test.shape[0], desc="Processing anime"):
    anime_id = row['anime_id']
    anime_synopsis = row['Synopsis']
    anime_name = row['Name']
    anime_genre = row['Genres']

    # Prepare input text for classification
    text = f"The anime is called {anime_name}. The synopsis of the anime is: {anime_synopsis}"

    # Predict genres using the fine-tuned model
    output = zeroshot_classifier(text, unique_genres, multi_label=True)

    # Extract predicted genres
    anime_genre_len = len(anime_genre.split(','))
    top_k_labels_predicted = output['labels'][0:anime_genre_len]

    """ print("Anime ID ", anime_id)
    print("Genre ", anime_genre.split(','))
    print("Top k labels predicted ", top_k_labels_predicted) """

    # Store results
    results[anime_id] = {}
    for i in range(anime_genre_len):
        genre = anime_genre.split(',')[i].lower()
        results[anime_id][genre] = "error"
        if genre in top_k_labels_predicted:
            results[anime_id][genre] = "correct"
    
    """ if len(results) == 10:
        break """

# Save the results to a JSON file
with open('results_finetuned_model.json', 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
import json
from tqdm import tqdm

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")  # change the model identifier here

genre_keywords = {
    "adventure": ["quest", "journey", "explore", "discovery", "expedition"],
    "sliceoflife": ["school", "daily life", "relationships", "emotions", "youth"],
    "action": ["fight", "battle", "combat", "war", "violence", "hero"],
    "romance": ["love", "passion", "relationship", "heart", "romantic"],
    "erotica": ["sensual", "explicit", "adult", "intimate"],
    "ecchi": ["fanservice", "sexy", "provocative"],
    "comedy": ["funny", "joke", "laugh", "humor", "silly"],
    "suspense": ["mystery", "thriller", "suspense", "tension", "cliffhanger"],
    "supernatural": ["ghost", "superpower", "vampire", "magic", "witch", "occult"],
    "fantasy": ["magic", "dragon", "wizard", "sword", "medieval", "mythical"],
    "hentai": ["adult", "explicit", "mature", "sexual", "forbidden"],
    "drama": ["conflict", "emotions", "heartbreak", "struggle", "family", "tragedy"],
    "sci-fi": ["space", "future", "alien", "robot", "science", "technology"],
    "boyslove": ["love", "romantic", "relationship", "boys"],
    "sports": ["game", "team", "competition", "sports", "training", "match"],
    "avantgarde": ["art", "experimental", "unique", "artsy", "abstract", "avant-garde"],
    "horror": ["scary", "monster", "ghost", "screaming", "fear", "gore"],
    "girlslove": ["love", "romance", "relationship", "girls"],
    "awardwinning": ["award", "nomination", "critically acclaimed"],
    "unknown": ["unknown", "mystery", "confusion"],
    "mystery": ["detective", "investigation", "puzzle", "whodunit", "crime"],
    "gourmet": ["food", "cooking", "restaurant", "chef", "gourmet", "culinary"],
}

# USAR TF-IDF PRA DESCOBRIR PALAVRAS??

def add_context_based_on_keywords(synopsis, genre_keywords):
    context = ""
    for genre, keywords in genre_keywords.items():
        if any(keyword.lower() in synopsis.lower() for keyword in keywords):
            # Add genre context if any keyword is found
            context += f" This anime involves {genre.lower()} elements, including {', '.join(keywords)}."

    # Return the augmented synopsis with additional context
    return synopsis + context

# Define genres
#genres = sorted(set(g.strip() for genre_list in data['Genres'].dropna() for g in genre_list.split(',')))

# Initialize results dictionary
results = {}

# Iterate over the test data with progress bar
for index, row in tqdm(test.iterrows(), total=test.shape[0], desc="Processing anime"):
    anime_id = row['anime_id']
    anime_synopsis = row['Synopsis']
    anime_name = row['Name']
    anime_genre = row['Genres']

    # Prepare input text for classification
    augmented_text = add_context_based_on_keywords(anime_synopsis, genre_keywords)

    text = 'The anime is called ' + anime_name + '. The synopsis of the anime is: ' + anime_synopsis + ". " + augmented_text

    # Predict genres using the fine-tuned model
    output = zeroshot_classifier(text, unique_genres, multi_label=True)

    # Extract predicted genres
    anime_genre_len = len(anime_genre.split(','))
    top_k_labels_predicted = output['labels'][0:anime_genre_len]

    print("Anime ID ", anime_id)
    print("Genre ", anime_genre.split(','))
    print("Top k labels predicted ", top_k_labels_predicted)

    # Store results
    results[anime_id] = {}
    for i in range(anime_genre_len):
        genre = anime_genre.split(',')[i].lower()
        results[anime_id][genre] = "error"
        if genre in top_k_labels_predicted:
            results[anime_id][genre] = "correct"
    
    if len(results) == 10:
        break

# Save the results to a JSON file
with open('results_context_classification.json', 'w') as f:
    json.dump(results, f, indent=4)